Берем отызывы за лето (из архива с материалами или предыдущего занятия)

1. Учим conv сеть для классификации
2. Рассмотреть 2-а варианта сеточек 
3. Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/
4. Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)
 				
Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше

In [1]:
# Импорт библиотек
import gensim
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")
from nltk.probability import FreqDist
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.callbacks import TensorBoard 
from keras.losses import SparseCategoricalCrossentropy
from keras.callbacks import EarlyStopping  
from sklearn.preprocessing import LabelEncoder

[nltk_data] Downloading package punkt to /home/dmitriy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2023-12-25 19:17:43.954235: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-25 19:17:44.118180: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-25 19:17:44.120040: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-25 19:17:44.986999: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
data_path = r'/home/dmitriy/Downloads/ai_nlp_data/hw_7/'

In [3]:
import keras.backend as K


def get_f1(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [4]:
df = pd.read_excel(os.path.join(data_path, r'отзывы за лето.xls'))

In [5]:
df_train, df_test = train_test_split(df, test_size=0.33, random_state=42)
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

### Предобработка данных

In [6]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()


def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)


df_train['Content'] = df_train['Content'].apply(preprocess_text)
df_test['Content'] = df_test['Content'].apply(preprocess_text)

In [7]:
train_corpus = " ".join(df_train["Content"])
train_corpus = train_corpus.lower()
tokens = word_tokenize(train_corpus)

In [8]:
tokens_filtered = [word for word in tokens if word.isalnum()]
dist = FreqDist(tokens_filtered)

In [9]:
max_words = 200
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [10]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [11]:
max_len = 40


def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [12]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["Content"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["Content"]], dtype=np.int32)

In [13]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(df_train['Rating']) 
test_enc_labels = le.transform(df_test['Rating'])

In [14]:
num_classes = 5
y_train = tf.keras.utils.to_categorical(train_enc_labels, num_classes=num_classes)
y_test = tf.keras.utils.to_categorical(test_enc_labels, num_classes=num_classes)

### Keras CONV модель с Embedding слоем по умолчанию

In [15]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len)) 
                    #inputdim -- размер словаря, outputdim -- длина вектора, input_length -- длина входной последовательности
                    #на вход: (батч, inputlen), на выходе: (батч, inputlen, outputdim)
model.add(Conv1D(128, 3))
                   #128 -- длина 1D-фильтра, шаг -- 3
                   #на выходе ([128/3], 128)
model.add(Activation("relu"))
                    #применяем функцию активации к выходу предыдущего слоя
                    #на выходе ([128/3], 128)
model.add(GlobalMaxPool1D())
                    #в каждой свертке оставляет максимальный элемент
                    #на выходе ([128/3], 1)
model.add(Dense(10))
               #10-количество выходов
model.add(Activation("relu"))
model.add(Dense(num_classes))
                #num_classes = 5 -- количество выходов
model.add(Activation('softmax'))
                #преобразуем вектор в рапределение вероятностей

In [16]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=[get_f1])

In [17]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  
epochs = 20
batch_size = 512
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 2s 39ms/step - loss: 1.3791 - get_f1: 0.0664 - val_loss: 1.0514 - val_get_f1: 0.7153
Epoch 2/20
25/25 [==============================] - 1s 33ms/step - loss: 0.9220 - get_f1: 0.7253 - val_loss: 0.8715 - val_get_f1: 0.7433
Epoch 3/20
25/25 [==============================] - 1s 33ms/step - loss: 0.7711 - get_f1: 0.7645 - val_loss: 0.7735 - val_get_f1: 0.7527
Epoch 4/20
25/25 [==============================] - 1s 31ms/step - loss: 0.7156 - get_f1: 0.7690 - val_loss: 0.7477 - val_get_f1: 0.7525
Epoch 5/20
25/25 [==============================] - 1s 31ms/step - loss: 0.6957 - get_f1: 0.7702 - val_loss: 0.7350 - val_get_f1: 0.7572
Epoch 6/20
25/25 [==============================] - 1s 32ms/step - loss: 0.6807 - get_f1: 0.7770 - val_loss: 0.7241 - val_get_f1: 0.7576
Epoch 7/20
25/25 [==============================] - 1s 32ms/step - loss: 0.6702 - get_f1: 0.7792 - val_loss: 0.7228 - val_get_f1: 0.7585
Epoch 8/20
25/25 [=======================

In [18]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test f1_score:', score[1])

14/14 [==============================] - 0s 9ms/step - loss: 0.6764 - get_f1: 0.7775


Test loss: 0.6763707995414734
Test f1_score: 0.7774753570556641


### Keras CONV модель с предобученным Embedding слоем

In [20]:
word_vectors = gensim.models.KeyedVectors.load_word2vec_format(os.path.join(r'/home/dmitriy/Downloads/ai_nlp_data/hw_7/', 'model.bin'), binary=True) .vectors 

In [21]:
word_vectors_matrix = [word_vectors[i][:128] for i in range(200)]

In [22]:
initializer = tf.keras.initializers.Constant(word_vectors_matrix)
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, embeddings_initializer =initializer, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [23]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=[get_f1])

In [24]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  

epochs = 20
batch_size = 512

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 2s 48ms/step - loss: 0.9119 - get_f1: 0.7174 - val_loss: 0.8372 - val_get_f1: 0.7323
Epoch 2/20
25/25 [==============================] - 1s 34ms/step - loss: 0.7483 - get_f1: 0.7523 - val_loss: 0.7750 - val_get_f1: 0.7457
Epoch 3/20
25/25 [==============================] - 1s 34ms/step - loss: 0.6960 - get_f1: 0.7664 - val_loss: 0.7581 - val_get_f1: 0.7564
Epoch 4/20
25/25 [==============================] - 1s 33ms/step - loss: 0.6596 - get_f1: 0.7772 - val_loss: 0.7468 - val_get_f1: 0.7566
Epoch 5/20
25/25 [==============================] - 1s 33ms/step - loss: 0.6357 - get_f1: 0.7850 - val_loss: 0.7385 - val_get_f1: 0.7577
Epoch 6/20
25/25 [==============================] - 1s 33ms/step - loss: 0.6137 - get_f1: 0.7932 - val_loss: 0.7500 - val_get_f1: 0.7540


In [25]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test f1_score:', score[1])

14/14 [==============================] - 0s 9ms/step - loss: 0.7159 - get_f1: 0.7632


Test loss: 0.7158985137939453
Test f1_score: 0.7631778717041016
